In [1]:
import os
import json
import sys
import pandas as pd

In [2]:
os.chdir('T://lab_research//RES-Folder-UPOD//Echo_label//E_ResearchData//2_ResearchData')

In [3]:
'''
{projects: [{"name": "ECHO", "id": 77, "cuis": "", "tuis": "", 
    "documents":[
        {
            "id": 1, 
            "name": "doc1", 
            "text": "text1",
            "annotations": [
                {
                    "id": 1,
                    "user": "user1", 
                    "cui": 1, 
                    "start": 1,
                    "end": 2, 
                    "value": "value1",
                    "validated": true,
                    "correct": true,
                    "deleted": false,
                    "alternative": false,
                    "killed": false,
                    "meta_anns": {
                        "Negation":{
                            "name": "Negation",
                            "validated": true,
                            "accuracy": 1.0,
                            "value": "Negated"
                        }
                    }
            ]
    ]}
]}
'''
base_dict = {"projects": [{"name": "ECHO_LABEL", "id": 777, "cuis": "", "tuis": "", "documents":[]}]}

In [4]:
# read in jsonl file put in list of dictionaries
with open('outdb_140423.jsonl', 'r') as f:
    data = f.readlines()
    data = [json.loads(line) for line in data]
data_with_span = [d for d in data if "spans" in d.keys()]

In [5]:
# unique labels
set([s['label'] 
     for d in data_with_span
       for s in d["spans"]])

{'lv_sys_func_hyperdynamic',
 'lv_sys_func_improved',
 'lv_sys_func_mild',
 'lv_sys_func_moderate',
 'lv_sys_func_normal',
 'lv_sys_func_severe',
 'lv_sys_func_unchanged',
 'lv_sys_func_unknown'}

In [6]:
len(data), len(data_with_span)

(5000, 4966)

In [16]:
_base_annotation = { "user": "BVE", 
                    "cui": 1, 
                    "start": 1,
                    "end": 2, 
                    "value": "value1",
                    "validated": True,
                    "correct": True,
                    "deleted": False,
                    "alternative": False,
                    "killed": False
                }

documents = []
for idx, d in enumerate(data_with_span):
    new_doc = {"id": idx,
               "name": f"doc_{idx}", 
               "text": d["text"], 
               "annotations":[]}
    for jdx, span in enumerate(d['spans']):
        base_annotation = _base_annotation.copy()
        base_annotation['id'] = jdx
        base_annotation['cui'] = jdx
        base_annotation['start'] = span['start']
        base_annotation['end'] = span['end']
        base_annotation['value'] = d["text"][span['start']:span['end']]
        base_annotation['meta_anns'] = {"LeftVentricleFunction":{
                                            "name": "LeftVentricleFunction",
                                            "validated": True,
                                            "accuracy": 1.0,
                                            "value": span['label']
                                        }
                                      }
        new_doc["annotations"].append(base_annotation)
    documents.append(new_doc)
base_dict['projects'][0]['documents'] = documents

In [8]:
# write out json file
with open('metacat_outdb_140423.json', 'w') as f:
    json.dump(base_dict, f)

In [9]:
# read in json file with pandas
df = pd.read_json('outdb_140423.jsonl', lines=True)
df.reset_index().rename(columns={'index':'name'})[['name', 'text']].to_csv('outdb_140423_medcat.csv', index=False)

In [19]:
base_dict['projects'][0]['documents'][14]

{'id': 14,
 'name': 'doc_14',
 'text': "Borderline dilated LV with mild LV dysfunction; 3D LVEF :47%. Diastolic dysfunction grade I(a); E/E'= 3-4. Dilated atria. Mild M. Normal AoV function.\r\nDilated RV  (RVEDD 44 mm); Normal RV function. Homograft in PV position with moderate stenosis; PA V2 max: 378 cm/sec; PA max PG: 57 mmHg; PA mean PG: 29 mmHg; (prev Vmax was 339 cm/sec on 7/10/2020); Mild PR, mild TR; Vmax 330cm/s IVC normal in size and collapsing well. Estimated RVSP 46 mmHg.\r\n\r\nConclusie: borderline gedilateerde LV en RV met redelijke functie. Pulmonalishomograft met matige PS en spoor PI. Langzaam toename gradient over PVR tov 2019-2020.",
 'annotations': [{'user': 'BVE',
   'cui': 0,
   'start': 19,
   'end': 46,
   'value': 'LV with mild LV dysfunction',
   'validated': True,
   'correct': True,
   'deleted': False,
   'alternative': False,
   'killed': False,
   'id': 0,
   'meta_anns': {'LeftVentricleFunction': {'name': 'LeftVentricleFunction',
     'validated': True